In [2]:

import pandas as pd
import numpy as np

import folium
import io
import os

import pathlib

import requests
import seaborn as sns
import scipy.odr as odr
import statsmodels.api as sm

import scipy
import scipy.optimize as op
from scipy.stats import linregress
from scipy.optimize import curve_fit
from scipy import stats

#from pandas.stats.api import ols
from datetime import date, timedelta
import datetime

import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from scipy.optimize import curve_fit

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib
import matplotlib as mpl
import matplotlib.cm as cm
%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 12, 10
rcParams['pdf.fonttype'] = 42
inline_rc = dict(plt.rcParams)

#from matplotlib.pyplot import cm 
import platform
import sys
import glob
import time
import folium
import pymannkendall as mk
#import windrose
#from windrose import WindroseAxes

import geopandas as gpd
#import geemap

from functools import reduce

from contextlib import closing
import csv

import pyproj
from pyproj import Proj

import math
import ee
import openet.refetgee

#import spotpy
#import simplekml

In [3]:

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code:  4/1AfJohXk9G65yDMsWsquOrPPqrRYbcvA5lZuGpI1uNxZedE48-xaYokmZrKY



Successfully saved authorization token.


In [5]:
print("Operating System " + platform.system() + " " + platform.release())
print("Python Version " + str(sys.version))
print("Pandas Version " + str(pd.__version__))
print("Numpy Version " + str(np.__version__))
print("Matplotlib Version " + str(mpl.__version__))
#print("Well Application Version " + str(wa.__version__))
print("Scipy Version " +str(scipy.__version__))
print (os.environ['CONDA_DEFAULT_ENV'])


Operating System Windows 10
Python Version 3.11.6 | packaged by conda-forge | (main, Oct  3 2023, 10:29:11) [MSC v.1935 64 bit (AMD64)]
Pandas Version 2.1.2
Numpy Version 1.26.0
Matplotlib Version 3.8.0
Scipy Version 1.11.3
pygis311v3


In [6]:
sys.path.append("G:/My Drive/Python/")

import dbconnect

https://github.com/WSWUP/RefET

https://github.com/woodcrafty/PyETo

https://wswup.github.io/pyWeatherQAQC/qaqc_setup.html#downloading-the-repository

In [7]:
# https://developers.google.com/earth-engine/tutorials/community/time-series-visualization-with-altair

def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=30,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    bestEffort:
      Optional; A Boolean indicator for whether to use a larger scale if the
      geometry contains too many pixels at the given scale for the operation
      to succeed.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.
    tileScale:
      Optional; A number representing the scaling factor used to reduce
      aggregation tile size; using a larger tileScale (e.g. 2 or 4) may enable
      computations that run out of memory with the default.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function


# Define a function to transfer feature properties to a dictionary.
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

# Read in Station Data

If no station data are available, see the Compile_Weather_Station_Data notebook.

In [8]:
studystats = gpd.read_file('station_info.gpkg', driver='GPKG', layer='stations')
studystats['station_id'] = studystats['station_id'].astype(np.int32)
studystats = studystats.set_index(['station_id'])
#studystats
# Add Wellington Station
well = pd.DataFrame({'primary_id':'US-UTW','network':'UFN','latitude':39.4453,'longitude': -110.7288,'name':'Wellington'},index=[1682])
well.index.name = 'station_id'

well_gdf = gpd.GeoDataFrame(well,geometry=gpd.points_from_xy(well['longitude'],well['latitude'],crs='WGS 84'))

studystats = pd.concat([studystats,well_gdf],axis=0)

# Pull GRIDMET Data

<table class="dd-table mdl-data-table mdl-js-data-table" data-upgraded=",MaterialDataTable"><thead><tr><th class="mdl-data-table__cell--non-numeric"> Name </th><th class="mdl-data-table__cell--non-numeric"> Description </th><th> Min* </th><th> Max* </th> <th> Units </th> </tr></thead><tbody><tr><td class="mdl-data-table__cell--non-numeric"> pr </td><td class="mdl-data-table__cell--non-numeric"> Precipitation amount </td><td> 0 </td><td> 690.44 </td><td> mm, daily total </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> rmax </td><td class="mdl-data-table__cell--non-numeric"> Maximum relative humidity </td><td> 1.05 </td><td> 100 </td><td> % </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> rmin </td><td class="mdl-data-table__cell--non-numeric"> Minimum relative humidity </td><td> 0 </td><td> 100 </td><td> % </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> sph </td><td class="mdl-data-table__cell--non-numeric"> Specific humididy </td><td> 0 </td><td> 0.02 </td><td> Mass fraction </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> srad </td><td class="mdl-data-table__cell--non-numeric"> Surface downward shortwave radiation </td><td> 0 </td><td> 455.61 </td><td> W/m^2 </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> th </td><td class="mdl-data-table__cell--non-numeric"> Wind direction </td><td>  </td><td>  </td><td> deg </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> tmmn </td><td class="mdl-data-table__cell--non-numeric"> Minimum temperature </td><td> 225.54 </td><td> 314.88 </td><td> K </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> tmmx </td><td class="mdl-data-table__cell--non-numeric"> Maximum temperature </td><td> 233.08 </td><td> 327.14 </td><td> K </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> vs </td><td class="mdl-data-table__cell--non-numeric"> Wind velocity at 10m </td><td> 0.14 </td><td> 29.13 </td><td> m/s </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> erc </td><td class="mdl-data-table__cell--non-numeric"> Energy release component </td><td> 0 </td><td> 131.85 </td><td> NFDRS fire danger index </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> eto </td><td class="mdl-data-table__cell--non-numeric"> Daily grass reference evapotranspiration </td><td> 0 </td><td> 17.27 </td><td> mm </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> bi </td><td class="mdl-data-table__cell--non-numeric"> Burning index </td><td> 0 </td><td> 214.2 </td><td> NFDRS fire danger index </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> fm100 </td><td class="mdl-data-table__cell--non-numeric"> 100-hour dead fuel moisture </td><td> 0.28 </td><td> 33.2 </td><td> % </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> fm1000 </td><td class="mdl-data-table__cell--non-numeric"> 1000-hour dead fuel moisture </td><td> 0.36 </td><td> 47.52 </td><td> % </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> etr </td><td class="mdl-data-table__cell--non-numeric"> Daily alfalfa reference evapotranspiration </td><td> 0 </td><td> 27.02 </td><td> mm </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> vpd </td><td class="mdl-data-table__cell--non-numeric"> Mean vapor pressure deficit </td><td> 0 </td><td> 9.83 </td><td> kPa </td></tr></tbody></table>

In [30]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-8, 'years'), today)
pdsi = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(date_range).select(['pr', 'rmax', 'rmin', 'sph', 'srad', 'th', 
                                                                                 'tmmn', 'tmmx', 'vs', 'eto', 'bi', 'etr', 'vpd'])
#aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
#    ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
                                                                                 
dataframes = {}

for station in studystats.index.unique():
    try:
        lon = pd.to_numeric(studystats.loc[station, 'longitude'])
        lat = pd.to_numeric(studystats.loc[station, 'latitude'])
        name = studystats.loc[station, 'name']
        
        aoi = ee.Geometry.Point(lon, lat)
    
        reduce_pdsi = create_reduce_region_function(geometry=aoi, reducer=ee.Reducer.mean(), scale=100, crs='EPSG:3310')
        
        pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(ee.Filter.notNull(pdsi.first().bandNames()))
        
        pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()
        
        pdsi_df = pd.DataFrame(pdsi_dict)
        pdsi_df['latitude'] = lat
        pdsi_df['longitude'] = lon
        pdsi_df['name'] = name
        pdsi_df['stationid'] = station
        pdsi_df = add_date_info(pdsi_df)
        pdsi_df = pdsi_df.groupby(['stationid',pd.Grouper(key='Timestamp',freq='1D')]).first()
        pdsi_df = pdsi_df.reset_index()[['eto', 'etr', 'pr', 'rmax', 'rmin', 'sph', 'srad','th', 'tmmn', 'tmmx', 'vpd', 'vs', 'stationid', 'Timestamp']]
        pdsi_df['datasource'] = 'gridmet'
        pdsi_df = pdsi_df.replace(0,np.nan)
        dataframes[station] = pdsi_df
        print(lon,lat,name)
    except:
        print(station)
        pass


-111.43722 39.134 Buck Flat
-111.45383 39.30229 GBRC Meadows
-111.30576 39.61774 Huntington Horse
-111.31818 39.68338 Mammoth-Cottonwood
-111.50527 39.13648 Mt Baldy
-111.27221 39.45197 Red Pine Ridge
-111.43297 39.31042 Seeley Creek
-111.2491 39.4155 Upper Joes Valley
-111.35685 39.13233 Wrigley Creek
-110.83798 39.421265 Elmo
-111.15426 39.075554 Ferron
-110.97652 39.30698 Huntington
-111.07003 39.22013 Orangeville
-111.11027 39.39874  Bear Valley Canyon
-111.322778 39.309444  JOES VALLEY
-111.15617 39.21345  North Horn
-111.019 39.22683  EW7368 Castle Dale
-111.32278 39.30944  JOES VALLEY
-110.7288 39.4453 Wellington


In [33]:
gridmet_dfs = pd.concat(dataframes)
gridmet = gridmet_dfs.reset_index()
gridmet = gridmet.drop(['level_0','level_1'],axis=1)
#gridmet_dfs.to_csv("gridmet_weather_station_data.csv")
#gridmet_dfs.reset_index().to_feather("gridmet_weather_station_data.feather")
gridmet.to_parquet("gridmet_weather_station_data.parquet")

In [34]:
gridmet_dfs = pd.read_parquet("gridmet_weather_station_data.parquet")
gridmet_dfs 

,eto,etr,pr,rmax,rmin,sph,srad,th,tmmn,tmmx,vpd,vs,stationid,Timestamp,datasource
0,0.595700,0.832598,0.915254,100.000000,50.046772,0.002209,60.355228,241.416672,258.515564,270.964905,0.105253,2.912609,1294660,2015-11-26,gridmet
1,0.534466,0.712670,2.047021,100.000000,49.907200,0.001972,83.631004,208.540085,258.315277,269.484467,0.103806,1.907655,1294660,2015-11-27,gridmet
2,0.563649,0.763269,0.580645,100.000000,43.009117,0.001832,108.699203,289.957336,255.735870,270.434875,0.118241,1.812036,1294660,2015-11-28,gridmet
3,0.406786,0.528354,1.968289,100.000000,51.643440,0.001675,59.179554,240.293518,254.935883,266.904938,0.072681,1.716100,1294660,2015-11-29,gridmet
4,0.876137,1.339630,NaN,84.367973,33.168739,0.001297,111.307526,274.625854,256.740784,269.181061,0.164465,4.035346,1294660,2015-11-30,gridmet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55399,1.400000,2.100000,0.500000,100.000000,41.799999,0.004360,105.800003,166.000000,269.899994,284.899994,0.340000,3.500000,1682,2023-11-18,gridmet
55400,1.500000,2.300000,NaN,83.699997,49.500000,0.004410,60.000000,298.000000,274.799988,282.700012,0.340000,4.500000,1682,2023-11-19,gridmet
55401,1.400000,2.200000,NaN,100.000000,35.900002,0.003310,103.400002,346.000000,267.600006,282.899994,0.360000,3.400000,1682,2023-11-20,gridmet
55402,0.700000,1.000000,NaN,100.000000,28.400000,0.002780,125.699997,113.000000,265.399994,283.500000,0.420000,0.600000,1682,2023-11-21,gridmet


# GPM

https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_MONTHLY_V06

In [12]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-3, 'years'), today)
pdsi = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").filterDate(date_range).select(['precipitation',
                                                                                          'probabilityLiquidPrecipitation'])
#aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
#    ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
                                                                                 
dataframes = {}

for station in studystats.index.unique():
    try:
    
        lon = pd.to_numeric(studystats.loc[station, 'longitude'])
        lat = pd.to_numeric(studystats.loc[station, 'latitude'])
        name = studystats.loc[station, 'name']
        
        aoi = ee.Geometry.Point(lon, lat)
    
        reduce_pdsi = create_reduce_region_function(geometry=aoi, reducer=ee.Reducer.mean(), scale=8000, crs='EPSG:3310')
        
        pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(ee.Filter.notNull(pdsi.first().bandNames()))
        
        pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()
        
        pdsi_df = pd.DataFrame(pdsi_dict)
        pdsi_df['latitude'] = lat
        pdsi_df['longitude'] = lon
        pdsi_df['name'] = name
        pdsi_df['stationid'] = station
        pdsi_df = add_date_info(pdsi_df)
        dataframes[station] = pdsi_df
        print(lon,lat,name)
    except:
        pass



-111.43722 39.134 Buck Flat
-111.45383 39.30229 GBRC Meadows
-111.30576 39.61774 Huntington Horse
-111.31818 39.68338 Mammoth-Cottonwood
-111.50527 39.13648 Mt Baldy
-111.27221 39.45197 Red Pine Ridge
-111.43297 39.31042 Seeley Creek
-111.2491 39.4155 Upper Joes Valley
-111.35685 39.13233 Wrigley Creek
-110.83798 39.421265 Elmo
-111.15426 39.075554 Ferron
-110.97652 39.30698 Huntington
-111.07003 39.22013 Orangeville
-111.11027 39.39874  Bear Valley Canyon
-111.322778 39.309444  JOES VALLEY
-111.15617 39.21345  North Horn
-111.019 39.22683  EW7368 Castle Dale
-111.32278 39.30944  JOES VALLEY
-110.7288 39.4453 Wellington


In [13]:
gpm_dfs = pd.concat(dataframes)
gpm = gpm_dfs.reset_index().rename(columns={'precipitation':'pr','probabilityLiquidPrecipitation':'prob_pr'})[['pr', 'prob_pr', 'stationid','Timestamp']]
gpm['datasource'] = 'gpm'
#gpm_dfs.to_csv("gpm_weather_station_data.csv")
#gpm_dfs.reset_index().to_feather("gpm_weather_station_data.feather")
gpm.to_parquet("gpm_weather_station_data.parquet")

# NLDAS

<table class="dd-table mdl-data-table mdl-js-data-table" data-upgraded=",MaterialDataTable"><thead><tr><th class="mdl-data-table__cell--non-numeric"> Name </th><th class="mdl-data-table__cell--non-numeric"> Description </th><th> Min* </th><th> Max* </th> <th> Units </th> </tr></thead><tbody><tr><td class="mdl-data-table__cell--non-numeric"> temperature </td><td class="mdl-data-table__cell--non-numeric"> Air temperature at 2 meters above the surface </td><td> -49.79 </td><td> 51.2 </td><td> &amp;deg;C </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> specific_humidity </td><td class="mdl-data-table__cell--non-numeric"> Specific humidity at 2 meters above the surface </td><td> 0 </td><td> 0.02 </td><td> Mass fraction </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> pressure </td><td class="mdl-data-table__cell--non-numeric"> Surface pressure </td><td> 61847.58 </td><td> 105337.62 </td><td> Pa </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> wind_u </td><td class="mdl-data-table__cell--non-numeric"> U wind component at 10 meters above the surface </td><td> -27.93 </td><td> 27.54 </td><td> m/s </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> wind_v </td><td class="mdl-data-table__cell--non-numeric"> V wind component at 10 meters above the surface </td><td> -27.45 </td><td> 35.13 </td><td> m/s </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> longwave_radiation </td><td class="mdl-data-table__cell--non-numeric"> Surface downward longwave radiation </td><td> 72.18 </td><td> 545.11 </td><td> W/m^2 </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> convective_fraction </td><td class="mdl-data-table__cell--non-numeric"> Fraction of total precipitation that is convective: from NARR </td><td> 0 </td><td> 1 </td><td>  </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> potential_energy </td><td class="mdl-data-table__cell--non-numeric"> Convective available potential energy (J/kg): from NARR </td><td> 0 </td><td> 76666.17 </td><td> J/kg </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> potential_evaporation </td><td class="mdl-data-table__cell--non-numeric"> Potential evaporation: from NARR </td><td> 0 </td><td> 2.76 </td><td> kg/m^2 </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> total_precipitation </td><td class="mdl-data-table__cell--non-numeric"> Hourly total precipitation </td><td> 0 </td><td> 124.19 </td><td> kg/m^2 </td></tr><tr><td class="mdl-data-table__cell--non-numeric"> shortwave_radiation </td><td class="mdl-data-table__cell--non-numeric"> Surface downward shortwave radiation - bias corrected </td><td> 0 </td><td> 1368.54 </td><td> W/m^2 </td></tr></tbody></table>

In [10]:
72/12

6.0

In [7]:
today = ee.Date(pd.to_datetime('today'))

#aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
#    ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
                                                                                 


for month in range(-72, 0, 6):
    dataframes = {}
    date_range = ee.DateRange(today.advance(month, 'months'), today.advance(month+6, 'months'))
    pdsi = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').filterDate(date_range).select(['temperature', 
                                                                                         'specific_humidity',
                                                                                         'wind_u','wind_v',
                                                                                         'longwave_radiation', 
                                                                                         'potential_evaporation',
                                                                                         'total_precipitation', 
                                                                                         'shortwave_radiation'])
    for station in studystats.index.unique():
        try:
            lon = pd.to_numeric(studystats.loc[station, 'longitude'])
            lat = pd.to_numeric(studystats.loc[station, 'latitude'])
            name = studystats.loc[station, 'name']
            
            aoi = ee.Geometry.Point(lon, lat)
        
            reduce_pdsi = create_reduce_region_function(geometry=aoi, reducer=ee.Reducer.mean(), scale=8000, crs='EPSG:3310')
            
            pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(ee.Filter.notNull(pdsi.first().bandNames()))
            
            pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()
            
            pdsi_df = pd.DataFrame(pdsi_dict)
            pdsi_df['latitude'] = lat
            pdsi_df['longitude'] = lon
            pdsi_df['name'] = name
            pdsi_df['stationid'] = station
            pdsi_df = add_date_info(pdsi_df)
            dataframes[f"{station}_{month}"] = pdsi_df
            
            print(lon,lat,name, month)
        except:
            print(station, month)
            pass
    print(month)
    pd.concat(dataframes).to_parquet(f"./nlads_weather_station_data_{month}.parquet")




-111.43722 39.134 Buck Flat -72
-111.45383 39.30229 GBRC Meadows -72
-111.30576 39.61774 Huntington Horse -72
-111.31818 39.68338 Mammoth-Cottonwood -72
-111.50527 39.13648 Mt Baldy -72
-111.27221 39.45197 Red Pine Ridge -72
-111.43297 39.31042 Seeley Creek -72
-111.2491 39.4155 Upper Joes Valley -72
-111.35685 39.13233 Wrigley Creek -72
-110.83798 39.421265 Elmo -72
-111.15426 39.075554 Ferron -72
-110.97652 39.30698 Huntington -72
-111.07003 39.22013 Orangeville -72
-111.11027 39.39874  Bear Valley Canyon -72
-111.322778 39.309444  JOES VALLEY -72
-111.15617 39.21345  North Horn -72
-111.019 39.22683  EW7368 Castle Dale -72
-111.32278 39.30944  JOES VALLEY -72
-110.7288 39.4453 Wellington -72
-72


C:\Users\pauli\anaconda3\envs\pygis311new\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


-111.43722 39.134 Buck Flat -66
-111.45383 39.30229 GBRC Meadows -66
-111.30576 39.61774 Huntington Horse -66
-111.31818 39.68338 Mammoth-Cottonwood -66
-111.50527 39.13648 Mt Baldy -66
-111.27221 39.45197 Red Pine Ridge -66
-111.43297 39.31042 Seeley Creek -66
-111.2491 39.4155 Upper Joes Valley -66
-111.35685 39.13233 Wrigley Creek -66
-110.83798 39.421265 Elmo -66
-111.15426 39.075554 Ferron -66
-110.97652 39.30698 Huntington -66
-111.07003 39.22013 Orangeville -66
-111.11027 39.39874  Bear Valley Canyon -66
-111.322778 39.309444  JOES VALLEY -66
-111.15617 39.21345  North Horn -66
-111.019 39.22683  EW7368 Castle Dale -66
-111.32278 39.30944  JOES VALLEY -66
-110.7288 39.4453 Wellington -66
-66


C:\Users\pauli\anaconda3\envs\pygis311new\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


-111.43722 39.134 Buck Flat -60
-111.45383 39.30229 GBRC Meadows -60
-111.30576 39.61774 Huntington Horse -60
-111.31818 39.68338 Mammoth-Cottonwood -60
-111.50527 39.13648 Mt Baldy -60
-111.27221 39.45197 Red Pine Ridge -60
-111.43297 39.31042 Seeley Creek -60
-111.2491 39.4155 Upper Joes Valley -60
-111.35685 39.13233 Wrigley Creek -60
-110.83798 39.421265 Elmo -60
-111.15426 39.075554 Ferron -60
-110.97652 39.30698 Huntington -60
-111.07003 39.22013 Orangeville -60
-111.11027 39.39874  Bear Valley Canyon -60
-111.322778 39.309444  JOES VALLEY -60
-111.15617 39.21345  North Horn -60
-111.019 39.22683  EW7368 Castle Dale -60
-111.32278 39.30944  JOES VALLEY -60
-110.7288 39.4453 Wellington -60
-60


...

C:\Users\pauli\anaconda3\envs\pygis311new\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


-111.43722 39.134 Buck Flat -12
-111.45383 39.30229 GBRC Meadows -12
-111.30576 39.61774 Huntington Horse -12
-111.31818 39.68338 Mammoth-Cottonwood -12
-111.50527 39.13648 Mt Baldy -12
-111.27221 39.45197 Red Pine Ridge -12
-111.43297 39.31042 Seeley Creek -12
-111.2491 39.4155 Upper Joes Valley -12
-111.35685 39.13233 Wrigley Creek -12
-110.83798 39.421265 Elmo -12
-111.15426 39.075554 Ferron -12
-110.97652 39.30698 Huntington -12
-111.07003 39.22013 Orangeville -12
-111.11027 39.39874  Bear Valley Canyon -12
-111.322778 39.309444  JOES VALLEY -12
-111.15617 39.21345  North Horn -12
-111.019 39.22683  EW7368 Castle Dale -12
-111.32278 39.30944  JOES VALLEY -12
-110.7288 39.4453 Wellington -12
-12


C:\Users\pauli\anaconda3\envs\pygis311new\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


-111.43722 39.134 Buck Flat -6
-111.45383 39.30229 GBRC Meadows -6
-111.30576 39.61774 Huntington Horse -6
-111.31818 39.68338 Mammoth-Cottonwood -6
-111.50527 39.13648 Mt Baldy -6
-111.27221 39.45197 Red Pine Ridge -6
-111.43297 39.31042 Seeley Creek -6
-111.2491 39.4155 Upper Joes Valley -6
-111.35685 39.13233 Wrigley Creek -6
-110.83798 39.421265 Elmo -6
-111.15426 39.075554 Ferron -6
-110.97652 39.30698 Huntington -6
-111.07003 39.22013 Orangeville -6
-111.11027 39.39874  Bear Valley Canyon -6
-111.322778 39.309444  JOES VALLEY -6
-111.15617 39.21345  North Horn -6
-111.019 39.22683  EW7368 Castle Dale -6
-111.32278 39.30944  JOES VALLEY -6
-110.7288 39.4453 Wellington -6
-6


C:\Users\pauli\anaconda3\envs\pygis311new\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [15]:
locdir= pathlib.Path("./")

nldas_pq = {}

for file in locdir.glob("*_data_-*.parquet"):
    version = str(file.name).split(".")[0].split("_")[-1]
    nldas_pq[version] = pd.read_parquet(file)

nldas_dfs = pd.concat(nldas_pq)


In [18]:


nldas = nldas_dfs.reset_index().rename(columns={'longwave_radiation':'lw', 
                                                'potential_evaporation':'pet',
                                                'shortwave_radiation':'sw', 
                                                'specific_humidity':'sp_hum',
                                                'temperature':'temp', 
                                                'total_precipitation':'pr',
                                               })[['lw','sw','pet','wind_u','wind_v',
                                                   'sp_hum','temp','pr','stationid', 'Timestamp']]
nldas['datasource'] = 'NLDAS'

def agg_nldas(x):
    d = {}
    d['lw'] = x['lw'].mean()
    d['sw'] = x['sw'].mean()
    d['pet'] = x['pet'].sum()
    d['pr'] = x['pr'].sum()
    d['wind_u'] = x['wind_u'].median()
    d['wind_v'] = x['wind_v'].median()
    return pd.Series(d, list(d.keys()))

nldas_dly = nldas.groupby(['stationid',pd.Grouper(key='Timestamp',freq='1D')]).apply(agg_nldas).reset_index()
nldas_dly['datasource'] = 'NLDAS'

nldas_dly
#nlads_dfs.reset_index().to_feather("C:/Users/paulinkenbrandt/Documents/Projects/ET/nlads_weather_station_data.feather")



,stationid,Timestamp,lw,sw,pet,pr,wind_u,wind_v,datasource
0,1,2017-11-24,228.880000,261.448000,0.9253,0.0000,5.100,-2.380,NLDAS
1,1,2017-11-25,219.862917,107.922125,2.3347,0.0000,1.840,-0.480,NLDAS
2,1,2017-11-26,221.038750,112.929833,2.5104,0.0000,-0.910,3.585,NLDAS
3,1,2017-11-27,224.819583,95.062500,3.3446,0.0019,0.210,6.175,NLDAS
4,1,2017-11-28,205.262083,112.481042,2.3217,0.0350,3.215,-3.700,NLDAS
...,...,...,...,...,...,...,...,...,...
41586,1294775,2023-11-17,207.715000,124.284042,1.7836,0.0000,-0.685,1.750,NLDAS
41587,1294775,2023-11-18,229.997500,99.879500,1.7853,0.2170,-0.975,2.780,NLDAS
41588,1294775,2023-11-19,266.340000,73.040458,1.4756,2.3643,3.770,1.405,NLDAS
41589,1294775,2023-11-20,215.124167,120.803875,2.0717,0.1036,4.360,-4.245,NLDAS


In [42]:
#nlads_dfs.to_csv("C:/Users/paulinkenbrandt/Documents/Projects/ET/nlads_weather_station_data.csv")

In [20]:

#nlads_dfs.to_csv("./nlads_weather_station_data.csv")
nldas.to_parquet("./nlads_weather_station_data.parquet")
nldas_dly.to_parquet("./nlads_daily_weather_station_data.parquet")
#nldas.to_parquet("C:/Users/paulinkenbrandt/Documents/Projects/ET/nlads_weather_station_data.parquet")

In [21]:
#nlads_dfs = pd.concat(dataframes)
#nlads_dfs.to_csv("nlads_weather_station_data_2023.csv")
#nlads_dfs.reset_index().to_feather("nlads_weather_station_data_2023.feather")
#nlads_dfs.to_parquet("nlads_weather_station_data_2023.parquet")

In [ ]:
import ee
import openet.refetgee

source_img = ee.Image('NASA/NLDAS/FORA0125_H002/A20150701_2000')
etr = openet.refetgee.Hourly.nldas(source_img).etr\
    .reduceRegion(reducer=ee.Reducer.first(),
                  geometry=ee.Geometry.Point(-118.77388, 39.4575),
                  scale=1000)\
    .getInfo()

print('ETr: {:.2f} mm'.format(float(etr['etr'])))

In [54]:
nlads_dfs

longwave_radiation         millis  potential_evaporation  \
1294660 0                  265.42  1597597200000                 0.7014   
        1                  286.38  1597600800000                 0.7014   
        2                  286.41  1597604400000                 1.0185   
        3                  286.45  1597608000000                 1.0185   
        4                  290.88  1597611600000                 1.0185   
...                           ...            ...                    ...   
1682    26175              330.40  1691827200000                 0.0662   
        26176              328.17  1691830800000                 0.0662   
        26177              328.20  1691834400000                 0.0554   
        26178              328.22  1691838000000                 0.0554   
        26179              351.20  1691841600000                 0.0554   

               pressure  shortwave_radiation  specific_humidity  \
1294660 0      73198.62              632.908           0.002802   
        1      73270.43              748.884           0.002776   
        2      73274.33              769.049           0.002766   
        3      73278.23              747.531           0.002757   
        4      73281.58              697.392           0.002747   
...                 ...                  ...                ...   
1682    26175  83831.30                0.000           0.007172   
        26176  83885.58                0.000           0.007133   
        26177  83923.71                0.000           0.007060   
        26178  83961.72                0.000           0.006987   
        26179  83999.21                0.000           0.006914   

                 system:index  temperature  total_precipitation  wind_u  \
1294660 0      A20200816_1700    22.250000                  0.0   -1.46   
        1      A20200816_1800    24.030000                  0.0   -1.77   
        2      A20200816_1900    24.600000                  0.0   -1.79   
        3      A20200816_2000    25.170000                  0.0   -1.82   
        4      A20200816_2100    25.740000                  0.0   -1.84   
...                       ...          ...                  ...     ...   
1682    26175  A20230812_0800    22.660000                  0.0    2.25   
        26176  A20230812_0900    22.150039                  0.0    1.95   
        26177  A20230812_1000    21.710000                  0.0    2.36   
        26178  A20230812_1100    21.260000                  0.0    2.76   
        26179  A20230812_1200    20.820000                  0.0    3.16   

               wind_v  latitude  longitude        name  stationid  \
1294660 0       -2.74   39.1340 -111.43722   Buck Flat    1294660   
        1       -2.89   39.1340 -111.43722   Buck Flat    1294660   
        2       -2.69   39.1340 -111.43722   Buck Flat    1294660   
        3       -2.49   39.1340 -111.43722   Buck Flat    1294660   
        4       -2.28   39.1340 -111.43722   Buck Flat    1294660   
...               ...       ...        ...         ...        ...   
1682    26175   -2.79   39.4453 -110.72880  Wellington       1682   
        26176   -2.96   39.4453 -110.72880  Wellington       1682   
        26177   -3.13   39.4453 -110.72880  Wellington       1682   
        26178   -3.30   39.4453 -110.72880  Wellington       1682   
        26179   -3.47   39.4453 -110.72880  Wellington       1682   

                        Timestamp  Year  Month  Day  DOY  
1294660 0     2020-08-16 17:00:00  2020      8   16  229  
        1     2020-08-16 18:00:00  2020      8   16  229  
        2     2020-08-16 19:00:00  2020      8   16  229  
        3     2020-08-16 20:00:00  2020      8   16  229  
        4     2020-08-16 21:00:00  2020      8   16  229  
...                           ...   ...    ...  ...  ...  
1682    26175 2023-08-12 08:00:00  2023      8   12  224  
        26176 2023-08-12 09:00:00  2023      8   12  224  
        26177 2023-08-12 10:00:00  2023      8  

# MODIS 16 Terra Net 8-day

In [25]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-6, 'years'), today)
pdsi = ee.ImageCollection("MODIS/061/MOD16A2").filterDate(date_range).select(['ET','LE','PET','PLE','ET_QC'])

#aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
#    ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
                                                                                 
dataframes = {}

for station in studystats.index.unique():
    try:
        lon = pd.to_numeric(studystats.loc[station, 'longitude'])
        lat = pd.to_numeric(studystats.loc[station, 'latitude'])
        name = studystats.loc[station, 'name']
        
        aoi = ee.Geometry.Point(lon, lat)
    
        reduce_pdsi = create_reduce_region_function(geometry=aoi, reducer=ee.Reducer.mean(), scale=100, crs='EPSG:3310')
        
        pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(ee.Filter.notNull(pdsi.first().bandNames()))
        
        pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()
        
        pdsi_df = pd.DataFrame(pdsi_dict)
        pdsi_df['latitude'] = lat
        pdsi_df['longitude'] = lon
        pdsi_df['name'] = name
        pdsi_df['stationid'] = station
        pdsi_df = add_date_info(pdsi_df)
        dataframes[station] = pdsi_df
        print(lon,lat,name)
    except:
        print(station)
        pass


-111.43722 39.134 Buck Flat
-111.45383 39.30229 GBRC Meadows
-111.30576 39.61774 Huntington Horse
-111.31818 39.68338 Mammoth-Cottonwood
-111.50527 39.13648 Mt Baldy
-111.27221 39.45197 Red Pine Ridge
-111.43297 39.31042 Seeley Creek
-111.2491 39.4155 Upper Joes Valley
-111.35685 39.13233 Wrigley Creek
-110.83798 39.421265 Elmo
-111.15426 39.075554 Ferron
-110.97652 39.30698 Huntington
-111.07003 39.22013 Orangeville
-111.11027 39.39874  Bear Valley Canyon
-111.322778 39.309444  JOES VALLEY
-111.15617 39.21345  North Horn
-111.019 39.22683  EW7368 Castle Dale
-111.32278 39.30944  JOES VALLEY
-110.7288 39.4453 Wellington


In [26]:
modet_dfs = pd.concat(dataframes)

modet = modet_dfs.reset_index().rename(columns={'ET':'aet', 'LE':'le', 'PET':'pet', 'PLE':'ple'})[['aet','le','pet','ple', 'stationid', 'Timestamp']]
modet[['aet','le','pet','ple']] = modet[['aet','le','pet','ple']]/8*0.10 # 8 day to 1 day; kg/m2 water = mm
modet['days_in_month'] = modet.set_index('Timestamp').index.days_in_month
modet = modet.groupby(['stationid',pd.Grouper(key='Timestamp',freq='1MS')]).mean(numeric_only=True).reset_index()
modet['datasource'] = 'MODIS'

for col in ['aet','le','pet','ple']:
    modet[col] = modet[col]*modet['days_in_month']
#modet_dfs.to_csv("modiset_weather_station_data.csv")
#modet_dfs.reset_index().to_feather("modiset_weather_station_data.feather")
modet.to_parquet("modiset_weather_station_data.parquet")


# Daymet

In [23]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-6, 'years'), today)
pdsi = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(date_range).select(['prcp','srad','swe','vp'])

#aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
#    ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
                                                                                 
dataframes = {}

for station in studystats.index.unique():
    try:
        lon = pd.to_numeric(studystats.loc[station, 'longitude'])
        lat = pd.to_numeric(studystats.loc[station, 'latitude'])
        name = studystats.loc[station, 'name']
        
        aoi = ee.Geometry.Point(lon, lat)
    
        reduce_pdsi = create_reduce_region_function(geometry=aoi, reducer=ee.Reducer.mean(), scale=100, crs='EPSG:3310')
        
        pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(ee.Filter.notNull(pdsi.first().bandNames()))
        
        pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()
        
        pdsi_df = pd.DataFrame(pdsi_dict)
        pdsi_df['latitude'] = lat
        pdsi_df['longitude'] = lon
        pdsi_df['name'] = name
        pdsi_df['stationid'] = station
        pdsi_df = add_date_info(pdsi_df)
        dataframes[station] = pdsi_df
        print(lon,lat,name)
    except:
        print(station)
        pass

daymet_dfs = pd.concat(dataframes)
daymet = daymet_dfs.reset_index().rename(columns={'prcp':'pr','srad':'sw','vp':'vpd'})[[ 'pr','sw','swe','vpd', 'stationid', 'Timestamp']]
daymet['datasource'] = 'daymet'

#daymet_dfs.to_csv("daymet_weather_station_data.csv")
#daymet_dfs.reset_index().to_feather("daymet_weather_station_data.feather")
daymet.to_parquet("daymet_weather_station_data.parquet")

-111.43722 39.134 Buck Flat
-111.45383 39.30229 GBRC Meadows
-111.30576 39.61774 Huntington Horse
-111.31818 39.68338 Mammoth-Cottonwood
-111.50527 39.13648 Mt Baldy
-111.27221 39.45197 Red Pine Ridge
-111.43297 39.31042 Seeley Creek
-111.2491 39.4155 Upper Joes Valley
-111.35685 39.13233 Wrigley Creek
-110.83798 39.421265 Elmo
-111.15426 39.075554 Ferron
-110.97652 39.30698 Huntington
-111.07003 39.22013 Orangeville
-111.11027 39.39874  Bear Valley Canyon
-111.322778 39.309444  JOES VALLEY
-111.15617 39.21345  North Horn
-111.019 39.22683  EW7368 Castle Dale
-111.32278 39.30944  JOES VALLEY
-110.7288 39.4453 Wellington


In [31]:
daymet_dfs = pd.read_parquet("daymet_weather_station_data.parquet")
daymet_dfs

millis  prcp        srad   swe system:index          vp  \
1294660 0     1510444800000  0.00  235.520004  0.00     20171112  487.140015   
        1     1510531200000  0.00  251.279999  0.00     20171113  576.599976   
        2     1510617600000  0.00  212.630005  0.00     20171114  543.650024   
        3     1510704000000  0.00  249.990005  0.00     20171115  521.880005   
        4     1510790400000  0.00  200.520004  0.00     20171116  599.830017   
...                     ...   ...         ...   ...          ...         ...   
1682    1870  1672099200000  6.98  125.510002  0.00     20221227  402.000000   
        1871  1672185600000  4.06   95.589996  0.00     20221228  492.350006   
        1872  1672272000000  0.00  226.559998  0.00     20221229  266.760010   
        1873  1672358400000  3.80  134.800003  3.80     20221230  341.730011   
        1874  1672444800000  5.70  111.800003  2.55     20221231  489.299988   

              latitude  longitude        name  stationid  Timestamp  Year  \
1294660 0      39.1340 -111.43722   Buck Flat    1294660 2017-11-12  2017   
        1      39.1340 -111.43722   Buck Flat    1294660 2017-11-13  2017   
        2      39.1340 -111.43722   Buck Flat    1294660 2017-11-14  2017   
        3      39.1340 -111.43722   Buck Flat    1294660 2017-11-15  2017   
        4      39.1340 -111.43722   Buck Flat    1294660 2017-11-16  2017   
...                ...        ...         ...        ...        ...   ...   
1682    1870   39.4453 -110.72880  Wellington       1682 2022-12-27  2022   
        1871   39.4453 -110.72880  Wellington       1682 2022-12-28  2022   
        1872   39.4453 -110.72880  Wellington       1682 2022-12-29  2022   
        1873   39.4453 -110.72880  Wellington       1682 2022-12-30  2022   
        1874   39.4453 -110.72880  Wellington       1682 2022-12-31  2022   

              Month  Day  DOY  
1294660 0        11   12  316  
        1        11   13  317  
        2        11   14  318  
        3        11   15  319  
        4        11   16  320  
...             ...  ...  ...  
1682    1870     12   27  361  
        1871     12   28  362  
        1872     12   29  363  
        1873     12   30  364  
        1874     12   31  365  

[33750 rows x 15 columns]

# TerraClimate

In [18]:
today = ee.Date(pd.to_datetime('today'))
date_range = ee.DateRange(today.advance(-6, 'years'), today)
pdsi = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").filterDate(date_range).select(['aet',
                                                                                      'def',
                                                                                      'pdsi',
                                                                                      'pet','pr','ro','soil','srad','swe'])

#aoi = ee.FeatureCollection('EPA/Ecoregions/2013/L3').filter(
#    ee.Filter.eq('na_l3name', 'Sierra Nevada')).geometry()
                                                                                 
dataframes = {}

for station in studystats.index.unique():

    lon = pd.to_numeric(studystats.loc[station, 'longitude'])
    lat = pd.to_numeric(studystats.loc[station, 'latitude'])
    name = studystats.loc[station, 'name']
    
    aoi = ee.Geometry.Point(lon, lat)

    reduce_pdsi = create_reduce_region_function(geometry=aoi, reducer=ee.Reducer.mean(), scale=500, crs='EPSG:3310')
    
    pdsi_stat_fc = ee.FeatureCollection(pdsi.map(reduce_pdsi)).filter(ee.Filter.notNull(pdsi.first().bandNames()))
    
    pdsi_dict = fc_to_dict(pdsi_stat_fc).getInfo()
    
    pdsi_df = pd.DataFrame(pdsi_dict)
    pdsi_df['latitude'] = lat
    pdsi_df['longitude'] = lon
    pdsi_df['name'] = name
    pdsi_df['stationid'] = station
    pdsi_df = add_date_info(pdsi_df)
    dataframes[station] = pdsi_df
    print(lon,lat,name)

terraclimate_dfs = pd.concat(dataframes)
terra = terraclimate_dfs.reset_index().rename(columns={'srad':'sw'})[['aet', 'def', 'pet', 'pr', 'ro', 'soil', 'sw', 'swe',  'stationid', 'Timestamp']]
terra['datasource'] = 'terra'
for col in terra.columns:
    if col in ['aet', 'def', 'pet', 'soil', 'sw']:
        terra[col] = pd.to_numeric(terra[col])*0.1
        terra[col] = terra[col].replace(0,np.nan)
    elif col in ['pr','ro','soil','swe']:
        terra[col] = pd.to_numeric(terra[col])
        terra[col] = terra[col].replace(0,np.nan)

-111.43722 39.134 Buck Flat
-111.45383 39.30229 GBRC Meadows
-111.30576 39.61774 Huntington Horse
-111.31818 39.68338 Mammoth-Cottonwood
-111.50527 39.13648 Mt Baldy
-111.27221 39.45197 Red Pine Ridge
-111.43297 39.31042 Seeley Creek
-111.2491 39.4155 Upper Joes Valley
-111.35685 39.13233 Wrigley Creek
-110.83798 39.421265 Elmo
-111.15426 39.075554 Ferron
-110.97652 39.30698 Huntington
-111.07003 39.22013 Orangeville
-111.11027 39.39874  Bear Valley Canyon
-111.322778 39.309444  JOES VALLEY
-111.15617 39.21345  North Horn
-111.019 39.22683  EW7368 Castle Dale
-111.32278 39.30944  JOES VALLEY
-110.7288 39.4453 Wellington


In [21]:

#terraclimate_dfs.to_csv("terraclimate_weather_station_data.csv")
#terraclimate_dfs.reset_index().to_feather("terraclimate_weather_station_data.feather")
terra.to_parquet("terraclimate_weather_station_data.parquet")

In [22]:
terraclimate_dfs = pd.read_parquet("terraclimate_weather_station_data.parquet")
terraclimate_dfs 

,aet,def,pet,pr,ro,soil,sw,swe,stationid,Timestamp,datasource
0,NaN,NaN,NaN,19,NaN,11.9,95.8,19.0,1294660,2017-12-01,terra
1,NaN,NaN,NaN,36,NaN,11.9,98.8,55.0,1294660,2018-01-01,terra
2,NaN,NaN,NaN,31,NaN,11.9,136.8,87.0,1294660,2018-02-01,terra
3,NaN,NaN,NaN,40,NaN,11.9,180.7,127.0,1294660,2018-03-01,terra
4,65.8,NaN,65.8,24,6.0,56.7,223.2,35.0,1294660,2018-04-01,terra
...,...,...,...,...,...,...,...,...,...,...,...
1154,17.3,162.5,179.7,18,1.0,3.3,246.6,NaN,1682,2022-08-01,terra
1155,25.2,117.6,142.8,26,1.0,3.2,221.7,NaN,1682,2022-09-01,terra
1156,15.4,72.2,87.6,16,1.0,3.0,166.8,NaN,1682,2022-10-01,terra
1157,6.6,32.8,39.4,7,NaN,2.9,107.5,NaN,1682,2022-11-01,terra


In [11]:
import refet

etr = refet.Hourly(
    tmean=91.80, ea=1.20 , rs=61.16, uz=3.33, zw=3, elev=1208.5,
    lat=39.4575, lon=-118.77388, doy=182, time=18, method='asce',
    input_units={'tmean': 'F', 'rs': 'Langleys', 'uz': 'mph', 'lat': 'deg'}
    ).etr()

print(f'ETr: {float(etr[0]):.2f} mm')

ETr: 0.72 mm


In [12]:
etr = refet.Daily(
    tmin=66.65, tmax=102.80, tdew=49.84, rs=674.07, uz=4.80,
    zw=3, elev=1208.5, lat=39.4575, doy=182, method='asce',
    input_units={'tmin': 'F', 'tmax': 'F', 'tdew': 'F', 'rs': 'Langleys',
                 'uz': 'mph', 'lat': 'deg'}
    ).etr()

print(f'ETr: {float(etr[0]):.2f} mm')

TypeError: Daily.__init__() got an unexpected keyword argument 'tdew'

In [ ]:
https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customSingleStationReport/daily/start_of_period/714:UT:SNTL%7Cid=%22%22%7Cname/-2778,0/WTEQ::value,SNWD::value,PREC::value,TOBS::value,TMAX::value,TMIN::value,TAVG::value

In [59]:
stationid = 714
url_stub = f"https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customSingleStationReport/daily/start_of_period/{stationid}:UT:SNTL%7Cid=%22%22%7Cname/-2778,0/WTEQ::value,SNWD::value,PREC::value,TOBS::value,TMAX::value,TMIN::value,TAVG::value"

In [61]:
print(url_stub)

https://wcc.sc.egov.usda.gov/reportGenerator/view_csv/customSingleStationReport/daily/start_of_period/714:UT:SNTL%7Cid=%22%22%7Cname/-2778,0/WTEQ::value,SNWD::value,PREC::value,TOBS::value,TMAX::value,TMIN::value,TAVG::value


In [13]:
import ee
import openet.refetgee

source_img = ee.Image('ECMWF/ERA5_LAND/HOURLY/20150701T20')
etr = openet.refetgee.Hourly.era5_land(source_img).etr\
    .reduceRegion(reducer=ee.Reducer.first(),
                  geometry=ee.Geometry.Point(-118.77388, 39.4575),
                  scale=1000)\
    .getInfo()

print('ETr: {:.2f} mm'.format(float(etr['etr'])))

ETr: 0.88 mm


In [ ]:
import ee
import openet.refetgee

source_img = ee.Image('ECMWF/ERA5_LAND/HOURLY/20150701T20')
etr = openet.refetgee.Hourly.era5_land(source_img).etr\
    .reduceRegion(reducer=ee.Reducer.first(),
                  geometry=ee.Geometry.Point(-118.77388, 39.4575),
                  scale=1000)\
    .getInfo()

print('ETr: {:.2f} mm'.format(float(etr['etr'])))

In [17]:
startDate = '2020-01-01'
endDate = '2020-01-02'

collection = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
    .filterDate(startDate, endDate)
    .map(penet.refetgee.Hourly.era5_land.etr\
    .reduceRegion(reducer=ee.Reducer.first(),
                  geometry=ee.Geometry.Point(-118.77388, 39.4575),
                  scale=1000))

IndentationError: unexpected indent (2078383945.py, line 5)

In [27]:
input_coll = ee.ImageCollection('NASA/NLDAS/FORA0125_H002')\
    .filterDate('2020-04-01', '2020-04-02')
    
# input_img = ee.Image(ee.ImageCollection(gridmet_coll.first()))
input_img = ee.Image('NASA/NLDAS/FORA0125_H002/A20180401_1800')


test_pnt = ee.Geometry.Point(-119.01, 39.01);

# Project latitude array to match source elevation grid exactly
elev_img = ee.Image("projects/eddi-noaa/nldas/elevation")
lat_img = ee.Image.pixelLonLat().select('latitude')\
    .reproject('EPSG:4326', elev_img.projection().getInfo()['transform'])
    
# Set the output crs and crsTransform to match the source images
# crs = input_img.projection().getInfo()['wkt']
crs = input_img.projection().getInfo()['crs']
geo = input_img.projection().getInfo()['transform']

def hourly_etr(img):
    return openet.refetgee.Hourly.nldas(img, elev=elev_img, lat=lat_img).etr.reduceRegion(reducer=ee.Reducer.first(),
                  geometry=test_pnt,
                  scale=1000)\
    .getInfo()
        #.copyProperties(img, ['system:time_start'])

etr = ee.ImageCollection(input_coll.map(hourly_etr))
    
print(etr)

EEException: A mapped function's arguments cannot be used in client-side operations